In [1]:
import pandas as pd
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve
from sklearn.metrics import RocCurveDisplay
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt
import matplotlib
import random
import os
import argparse
from sklearn.metrics import precision_recall_curve

matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42 

dnase_preds="/mnt/lab_data2/anusri/chrombpnet/results/chrombpnet/DNASE_SE/GM12878/nautilus_runs/GM12878_03.06.2022_bias_128_4_1234_0.8_fold_0/dsqtl_preds/snp_scores.tsv"
enformer_dnase_preds="/mnt/lab_data2/anusri/chrombpnet/results/chrombpnet/DNASE_SE/GM12878/nautilus_runs/GM12878_03.06.2022_bias_128_4_1234_0.8_fold_0/dsqtl_preds_enformer/snp_scores.tsv"
atac_preds="/mnt/lab_data2/anusri/chrombpnet/results/chrombpnet/ATAC_PE/GM12878/nautilus_runs/GM12878_03.01.2022_bias_128_4_1234_0.4_fold_0/dsqtl_preds/snp_scores.tsv"

In [2]:
dsqtls_dnase = pd.read_csv(dnase_preds,sep="\t", header=0, index_col=False)
dsqtls_dnase[['gkm_SVM','kmer_SVM','abs_gkm_SVM','abs_kmer_SVM','CADD','GWAVA_region','GWAVA_tss','GWAVA_unmatched','GERP','label']] = dsqtls_dnase['META_DATA'].str.split(",", expand=True)

enformer_dsqtls_dnase = pd.read_csv(enformer_dnase_preds,sep="\t", header=0, index_col=False)
enformer_dsqtls_dnase[['gkm_SVM','kmer_SVM','abs_gkm_SVM','abs_kmer_SVM','CADD','GWAVA_region','GWAVA_tss','GWAVA_unmatched','GERP','label']] = dsqtls_dnase['META_DATA'].str.split(",", expand=True)

dsqtls_atac = pd.read_csv(atac_preds,sep="\t", header=0, index_col=False)
dsqtls_atac[['gkm_SVM','kmer_SVM','abs_gkm_SVM','abs_kmer_SVM','CADD','GWAVA_region','GWAVA_tss','GWAVA_unmatched','GERP','label']] = dsqtls_atac['META_DATA'].str.split(",", expand=True)

dsqtls_atac.shape

(28308, 18)

In [3]:
enformer_dsqtls_dnase["label"] = enformer_dsqtls_dnase["label"].astype(int)
enformer_dsqtls_dnase["abs_gkm_SVM"] = enformer_dsqtls_dnase["abs_gkm_SVM"].astype(float)
enformer_dsqtls_dnase["rsid"] = enformer_dsqtls_dnase["CHR"] + "_" + enformer_dsqtls_dnase["POS0"].astype(str) + "_" + enformer_dsqtls_dnase["REF"]+ "_" + enformer_dsqtls_dnase["ALT"]
#tts = enformer_dsqtls_dnase["rsid"]

dsqtls_atac["label"] = dsqtls_atac["label"].astype(int)
dsqtls_atac["abs_gkm_SVM"] = dsqtls_atac["abs_gkm_SVM"].astype(float)
dsqtls_atac["rsid"] = dsqtls_atac["CHR"] + "_" + dsqtls_atac["POS0"].astype(str) + "_" + dsqtls_atac["REF"]+ "_" + dsqtls_atac["ALT"]
#outs = set(dsqtls_atac["rsid"])
dsqtls_atac = dsqtls_atac[dsqtls_atac["rsid"].isin(enformer_dsqtls_dnase["rsid"])]

dsqtls_dnase["label"] = dsqtls_dnase["label"].astype(int)
dsqtls_dnase["abs_gkm_SVM"] = dsqtls_dnase["abs_gkm_SVM"].astype(float)
dsqtls_dnase["rsid"] = dsqtls_dnase["CHR"] + "_" + dsqtls_dnase["POS0"].astype(str) + "_" + dsqtls_dnase["REF"]+ "_" + dsqtls_dnase["ALT"]
dsqtls_dnase = dsqtls_dnase[dsqtls_dnase["rsid"].isin(enformer_dsqtls_dnase["rsid"])]



In [4]:
#outs = dsqtls_atac["rsid"]
#dsqtls_atac.shape

In [5]:
#tts[~tts.isin(outs)]

In [6]:
#outs[outs.isin(["chr10_555530_T_C"])]

In [7]:
#dsqtl=dsqtls_dnase
#dsqtl=dsqtls_atac
dsqtl=enformer_dsqtls_dnase
roc_auc = metrics.average_precision_score(dsqtl["label"], abs(dsqtl["log_counts_diff"]))
print(roc_auc)

0.31667568927503953


In [8]:
enformer_dsqtls_dnase.head()

,CHR,POS0,REF,ALT,META_DATA,log_counts_diff,log_probs_diff_abs_sum,probs_jsd_diff,gkm_SVM,kmer_SVM,abs_gkm_SVM,abs_kmer_SVM,CADD,GWAVA_region,GWAVA_tss,GWAVA_unmatched,GERP,label,rsid
0,chr1,846445,A,G,"2.653531,7.938784,2.653531,7.938784,-1.522533,...",-0.003076,-5.439789,-0.015283,2.653531,7.938784,2.653531,7.938784,-1.522533,0.48,0.53,0.75,-0.682,1,chr1_846445_A_G
1,chr1,901457,G,A,"2.821422,6.027648,2.821422,6.027648,1.739152,0...",0.007380,8.516491,0.020431,2.821422,6.027648,2.821422,6.027648,1.739152,0.29,0.36,0.89,2.15,1,chr1_901457_G_A
2,chr1,1176364,T,A,"-7.167236,-10.870018,7.167236,10.870018,-0.091...",0.024771,26.656536,0.049898,-7.167236,-10.870018,7.167236,10.870018,-0.091378,0.42,0.49,0.66,0.451,1,chr1_1176364_T_A
3,chr1,1217274,A,G,"-3.957382,2.387491,3.957382,2.387491,1.127472,...",0.013646,20.577271,0.049943,-3.957382,2.387491,3.957382,2.387491,1.127472,0.34,0.43,0.49,2.43,1,chr1_1217274_A_G
4,chr1,1580437,A,G,"-4.785596,-6.827102,4.785596,6.827102,0.665464...",0.047200,39.675499,0.034532,-4.785596,-6.827102,4.785596,6.827102,0.665464,0.38,0.35,0.86,1.64,1,chr1_1580437_A_G


In [9]:
dsqtls_dnase['abs_log_counts_diff'] = np.abs(dsqtls_dnase['log_counts_diff'])
dsqtls_atac['abs_log_counts_diff'] = np.abs(dsqtls_atac['log_counts_diff'])
enformer_dsqtls_dnase['abs_log_counts_diff'] = np.abs(enformer_dsqtls_dnase['log_counts_diff'])


In [10]:
(np.sum(dsqtls_atac.sort_values('abs_log_counts_diff')["label"][-1000:]==1)*100)/np.sum(dsqtls_dnase["label"]==1)




54.70383275261324

In [11]:
dsqtls_dnase.sort_values('log_counts_diff')

,CHR,POS0,REF,ALT,META_DATA,log_counts_diff,log_probs_diff_abs_sum,probs_jsd_diff,gkm_SVM,kmer_SVM,abs_gkm_SVM,abs_kmer_SVM,CADD,GWAVA_region,GWAVA_tss,GWAVA_unmatched,GERP,label,rsid,abs_log_counts_diff
219,chr17,39841835,C,T,"7.027461,4.983462,7.027461,4.983462,-0.272918,...",-1.722216,-474.85280,-0.151711,7.027461,4.983462,7.027461,4.983462,-0.272918,0.42,0.15,0.22,-4.7,1,chr17_39841835_C_T,1.722216
359,chr20,48019364,T,C,"18.674545000000002,12.399034,18.67454500000000...",-1.529098,-704.08466,-0.237357,18.674545000000002,12.399034,18.674545,12.399034,1.444861,0.36,0.48,0.45,0.675,1,chr20_48019364_T_C,1.529098
65,chr10,87974436,T,C,"13.422075,14.184656,13.422075,14.184656,0.1809...",-1.492431,-554.19750,-0.239231,13.422075,14.184656,13.422075,14.184656,0.180948,0.41,0.33,0.11,1.64,1,chr10_87974436_T_C,1.492431
127,chr13,40984793,T,C,"13.789686999999999,17.423902,13.78968699999999...",-1.465725,-411.41016,-0.152679,13.789686999999999,17.423902,13.789687,17.423902,0.38729,0.38,0.3,0.2,0.5920000000000001,1,chr13_40984793_T_C,1.465725
173,chr15,67961462,T,C,"11.228525,6.762428,11.228525,6.762428,1.538784...",-1.450597,-326.35846,-0.106155,11.228525,6.762428,11.228525,6.762428,1.538784,0.52,0.5,0.32,0.469,1,chr15_67961462_T_C,1.450597
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
542,chr8,71090360,C,T,"-3.792464,-1.597037,3.792464,1.597037,-0.21660...",0.971164,329.69293,0.105876,-3.792464,-1.597037,3.792464,1.597037,-0.21660900000000002,0.41,0.45,0.29,1.02,1,chr8_71090360_C_T,0.971164
199,chr16,83098673,C,T,"-8.216525,-4.767459,8.216525,4.767459,-0.24286...",1.004403,462.24463,0.172244,-8.216525,-4.767459,8.216525,4.767459,-0.242864,0.25,0.3,0.43,0.158,1,chr16_83098673_C_T,1.004403
196,chr16,55462750,T,C,"-8.042489999999999,-4.706761,8.042489999999999...",1.011383,320.06018,0.101352,-8.042489999999999,-4.706761,8.042490,4.706761,-0.07514900000000001,0.39,0.52,0.58,-2.12,1,chr16_55462750_T_C,1.011383
570,chr9,134981039,G,A,"-10.95599,-11.360225999999999,10.95599,11.3602...",1.573449,574.51670,0.209351,-10.95599,-11.360225999999999,10.955990,11.360225999999999,0.012338,0.44,0.4,0.59,-5.6,1,chr9_134981039_G_A,1.573449
